<a href="https://colab.research.google.com/github/tutpravitslon/applied_mechanics/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> **$\text{Exploratory Data Analysis}$** </center>
----

## $\texttt{Оглавление}$
1. [$\texttt{Проработка проблематики (в чем смысл для бизнеса)}$](#problems)
2. [$\texttt{Формализация задачи}$](#problem_statement)
3. [$\texttt{Анализ имеющихся данных и оценка их пригодности для решения поставленной задачи}$](#validity)
4. [$\texttt{Первичные разведочный анализ данных (EDA)}$](#eda)

# <a id="problems"> </a>$\text{Проблематика (в чем смысл для бизнеса)}$

<div class="alert alert-danger" role="alert">
Добавлена проработка проблематики для бизнеса.
</div>

Польза рекомендательных систем для пользователя:
- ускорение поиска конкретного товара
- дополниение аксессуаров к уже выбранным товарам
- предложение неожиданных товаров
- поддержка в принятии решения
- снижение информационного перегруза
- улучшение пользовательского опыта

Польза рекомендательных систем для бизнеса:
- Привлечение пользователей
- Увеличение целевых бизнес метрик
- Увеличение лояльности пользователей (NPS)
- Конкурентное преимущество
- Оптимизация ассортимента товаров

А также важная задача рекомендательных систем для бизнеса -  увеличение среднего чека.

# <a id="problem_statement"> </a>$\text{Формализация задачи}$

<div class="alert alert-danger" role="alert">
Исправлена ошибка: цель предсказать последовательность последующих покупок.
</div>

<div class="alert alert-info">
Прогназирование следующего товара, представляющего интерес для пользователя с предоставлением истории его взаимодействия.
</div>

- ассортимент товаров $I$, как множество всех возможных товаров;
- история $X_u$ для пользователя $U$, содержащая последовательность товаров $x_0, x_1, ..., x_{l-1} \in I$, с которыми $U$ до сих пор взаимодействовал.

> Цель рекомендательной системы: передсказать последовательность элементов, с которым пользователь провзаимодействует $x_l = y_{u^+}$. Набор последовательностей взаимодействия обозначим как набор данных $\mathscr{D}$, он состоить из $N$ пар пользавательской последовательности и целевых элементов  $\{(X_u, y_{u^+}), for \; u = 1, 2 ... , N\}$

# <a id="validity"> </a>Анализ имеющихся данных и оценка их пригодности для решения поставленной задачи

Данные представляют собой две связанные по `nm_id` таблицы, а также папку с изображениями. Имена изображений - `nm_id` + `.jpg`

В одной таблице содержится *Clickstream* для покупок пользователей, в другой таблице - некоторая информация о товаре.

<div class="alert alert-info">
Структура данных позволяет говорить о возможности построения системы  с персональными рекомендациями, неперсональными и гибридными. (Для пресональных рекомендаций можно использовать историю покупок пользователей)
</div>

<div class="alert alert-info">
Данные не содержат оценок пользователей. Фитбек неявный.
</div>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit


spark = SparkSession.builder \
    .appName("Process Characteristics") \
    .getOrCreate()

data_path = "/content/drive/MyDrive/wb_data/text_data_69020_final.parquet"
df = spark.read.parquet(data_path)

df.printSchema()


root
 |-- title: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- nm_id: long (nullable = true)
 |-- colornames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- characteristics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- charcID: double (nullable = true)
 |    |    |-- charcName: string (nullable = true)
 |    |    |-- charcSort: double (nullable = true)
 |    |    |-- charcType: double (nullable = true)
 |    |    |-- charcValues: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- groupID: double (nullable = true)
 |    |    |-- groupName: string (nullable = true)
 |    |    |-- groupSort: double (nullable = true)
 |    |    |-- isUnifying: boolean (nullable = true)
 |    |    |-- isVariable: boolean (nullable = true)
 |    |    |-- unitName: string (nullable = true)
 |    |    |-- value: double (nullable = true)
 |    |    |-- valueAuroraIDs: ar

In [ ]:
from pyspark.sql import functions as F

non_null_counts = df.select([F.count(F.when(F.col(c).isNotNull(), 1)).alias(c) for c in df.columns])

non_null_counts.show()

+------+---------+------+----------+---------------+-----------+
| title|brandname| nm_id|colornames|characteristics|description|
+------+---------+------+----------+---------------+-----------+
|417877|   413434|419766|    415774|         419766|     388463|
+------+---------+------+----------+---------------+-----------+



In [ ]:
from pyspark.sql import functions as F


# Разворачиваем массив characteristics в отдельные строки
df_exploded = df.withColumn("characteristic", F.explode(df["characteristics"]))

# Выбираем нужные поля из структуры characteristic и создаем новую таблицу
df_characteristics = df_exploded.select(
    "nm_id",  # Ссылка на nm_id для связи с основным DataFrame
    "characteristic.charcID",
    "characteristic.charcName",
    "characteristic.charcSort",
    "characteristic.charcType",
    "characteristic.charcValues",
    "characteristic.groupID",
    "characteristic.groupName",
    "characteristic.groupSort",
    "characteristic.isUnifying",
    "characteristic.isVariable",
    "characteristic.unitName",
    "characteristic.value",
    "characteristic.valueAuroraIDs",
    "characteristic.valueIDs",
    "characteristic.valueNames",
    "characteristic.variableValueIDs",
    "characteristic.variableValues",
    "characteristic.visibility"
)

df_characteristics_filtered = df_characteristics.where(df_characteristics.nm_id == 33500)

df_characteristics_filtered.show(df_characteristics_filtered.count(), truncate=False)


+-----+-------+----------------------------------------------+---------+---------+-------------------------------------------------+-------+---------+---------+----------+----------+--------+-----+--------------+--------+----------+----------------+--------------+----------+
|nm_id|charcID|charcName                                     |charcSort|charcType|charcValues                                      |groupID|groupName|groupSort|isUnifying|isVariable|unitName|value|valueAuroraIDs|valueIDs|valueNames|variableValueIDs|variableValues|visibility|
+-----+-------+----------------------------------------------+---------+---------+-------------------------------------------------+-------+---------+---------+----------+----------+--------+-----+--------------+--------+----------+----------------+--------------+----------+
|33500|NULL   |Рост модели на фото                           |NULL     |NULL     |NULL                                             |NULL   |NULL     |NULL     |NULL      |N

In [ ]:
#  Поиск выбросов категориальных данных пол осуществлялся эмпрерически
df_filtered = df_characteristics.filter(
    (df_characteristics.charcName == "Пол") &
    # (~F.array_contains(df_characteristics.charcValues, "Женщина")) &
    # (~F.array_contains(df_characteristics.charcValues, "Женский")) &
    (F.array_contains(df_characteristics.charcValues, "女")) # что в переводе с китайского женщина
    # (~F.array_contains(df_characteristics.charcValues, "Women")) &
    # (~F.array_contains(df_characteristics.charcValues, "женский")) &
    # (~F.array_contains(df_characteristics.charcValues, "женщина"))

)

df_filtered.show(truncate=False)


+------+-------+---------+---------+---------+-----------+-------+-------------------+---------+----------+----------+--------+-----+--------------+--------+----------+----------------+--------------+----------+
|nm_id |charcID|charcName|charcSort|charcType|charcValues|groupID|groupName          |groupSort|isUnifying|isVariable|unitName|value|valueAuroraIDs|valueIDs|valueNames|variableValueIDs|variableValues|visibility|
+------+-------+---------+---------+---------+-----------+-------+-------------------+---------+----------+----------+--------+-----+--------------+--------+----------+----------------+--------------+----------+
|66368 |NULL   |Пол      |23.0     |1.0      |[女]       |NULL   |Основная информация|1.0      |false     |false     |NULL    |NULL |NULL          |NULL    |[女]      |NULL            |NULL          |false     |
|217041|NULL   |Пол      |NULL     |NULL     |[女]       |NULL   |NULL               |NULL     |NULL      |NULL      |NULL    |NULL |NULL          |NULL   

In [ ]:

result = df.filter(df.nm_id == 66368)

result.show(truncate=False)  # truncate=False, чтобы выводить все данные в колонках

+-------------------------+-------------+-----+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_characteristics.groupBy("charcName").count().orderBy(F.desc("count")).show()
df_characteristics.groupBy("groupName").count().orderBy(F.desc("count")).show()

+--------------------+------+
|           charcName| count|
+--------------------+------+
|                 Пол|419766|
|               ТНВЭД|417109|
|                Цвет|415266|
|     Высота упаковки|397111|
|      Длина упаковки|397090|
|     Ширина упаковки|397083|
|              Состав|386872|
|          Назначение|385965|
| Страна производства|384442|
|              Покрой|372873|
|        Комплектация|372441|
|          Тип рукава|369430|
|     Вырез горловины|367407|
|        Вид застежки|365173|
|           Коллекция|346452|
|   Фактура материала|342927|
|      Уход за вещами|342671|
|        Тип карманов|341318|
|Декоративные элем...|326080|
|       Модель платья|325847|
+--------------------+------+
only showing top 20 rows

+--------------------+-------+
|           groupName|  count|
+--------------------+-------+
|                NULL|7933150|
| Основная информация|1216191|
|            Габариты|1188137|
|           Документы| 546055|
|                Цены|  89707|
|Допол

In [ ]:
df_characteristics.groupBy("charcName").agg(
    F.countDistinct("value").alias("unique_values_count"),
    F.count("*").alias("total_count")
).orderBy(F.desc("total_count")).show()

+--------------------+-------------------+-----------+
|           charcName|unique_values_count|total_count|
+--------------------+-------------------+-----------+
|                 Пол|                  0|     419766|
|               ТНВЭД|                  0|     417109|
|                Цвет|                  0|     415266|
|     Высота упаковки|                134|     397111|
|      Длина упаковки|                130|     397090|
|     Ширина упаковки|                114|     397083|
|              Состав|                  0|     386872|
|          Назначение|                  0|     385965|
| Страна производства|                  0|     384442|
|              Покрой|                  0|     372873|
|        Комплектация|                  0|     372441|
|          Тип рукава|                  0|     369430|
|     Вырез горловины|                  0|     367407|
|        Вид застежки|                  0|     365173|
|           Коллекция|                  0|     346452|
|   Фактур

In [ ]:
import pandas as pd

train_data_10_10_24_10_11_24_final = pd.read_parquet('/content/drive/MyDrive/wb_data/train_data_10_10_24_10_11_24_final.parquet')
train_data_10_10_24_10_11_24_final['dt'] = pd.to_datetime(train_data_10_10_24_10_11_24_final['dt'])

print(train_data_10_10_24_10_11_24_final['dt'].dtype)  # должен быть datetime64[ns]

print(train_data_10_10_24_10_11_24_final['dt'].head())


datetime64[ns]
0   2024-10-11 09:48:23
1   2024-10-10 19:44:54
2   2024-10-10 20:13:01
3   2024-10-10 19:27:19
4   2024-10-11 23:17:59
Name: dt, dtype: datetime64[ns]


In [ ]:
clicks_per_user_item = train_data_10_10_24_10_11_24_final.groupby(['wbuser_id', 'nm_id']).size().reset_index(name='clicks')

print(clicks_per_user_item.head())


   wbuser_id  nm_id  clicks
0          1  17528       4
1          1  31178       1
2          1  32385       1
3          1  43287       2
4          1  57951       1


In [ ]:
user_item_counts = train_data_10_10_24_10_11_24_final.groupby('wbuser_id')['nm_id'].nunique()


distribution = user_item_counts.value_counts().sort_index()



In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
    x=distribution.index,
    y=distribution.values,
    labels={"x": "Количество уникальных nm_id", "y": "Количество пользователей"},
    title=r"$ \text{Распределение пользователей по количеству взаимодействий с уникальными товарами}$"
)


fig.update_traces(marker=dict(size=10, color='blue', opacity=0.7))
fig.update_layout(
    xaxis=dict(title=r"$ \text{Количество уникальных nm_id}$"),
    yaxis=dict(title=r"$ \text{Количество пользователей}$"),
    template="plotly_white"
)

fig.show()

In [ ]:
clicks_per_item = train_data_10_10_24_10_11_24_final.groupby("nm_id")["wbuser_id"].nunique().reset_index()
clicks_per_item.columns = ["nm_id", "unique_clicks"]

clicks_per_item = clicks_per_item.sort_values(by="unique_clicks", ascending=False)


In [ ]:
clicks_distribution = clicks_per_item.groupby("unique_clicks").size().reset_index(name="count_nm_id")

clicks_distribution = clicks_distribution.sort_values(by="unique_clicks", ascending=False)


In [ ]:
# Построение графика
fig = px.bar(
    clicks_distribution,
    x="unique_clicks",  # Ось X — количество уникальных кликов
    y="count_nm_id",    # Ось Y — количество товаров
    title="Распределение товаров по количеству уникальных кликов",
    labels={"unique_clicks": "Уникальные клики", "count_nm_id": "Количество товаров"},
    text="count_nm_id"  # Отображение чисел над столбцами
)

# Настройка внешнего вида
fig.update_traces(texttemplate='%{text}', textposition='outside')  # Числа над столбцами
fig.update_layout(
    xaxis_type='log',  # Логарифмическая шкала на X, если данные сильно разрежены
    yaxis_type='log',  # Логарифмическая шкала на Y
    xaxis_title="Уникальные клики",
    yaxis_title="Количество товаров",
    title_font_size=20
)

fig.show()

In [ ]:

clicks_per_user_item = train_data_10_10_24_10_11_24_final.groupby(["wbuser_id", "nm_id"]).size().reset_index(name="click_count")

# Фильтруем пользователей, которые кликнули более 30 раз на один товар
clicks_over_30 = clicks_per_user_item[clicks_per_user_item["click_count"] > 30]

users_with_high_clicks = clicks_over_30["wbuser_id"].unique()

print(users_with_high_clicks)

[    315     675     763    1266    1854    2937    4307    9219   10782
   11218   11232   11348   11740   14478   15788   20377   20394   20871
   23110   23327   24055   31859   33403   36875   37153   37545   48457
   49044   51451   51960   52417   53473   54972   57101   57572   58759
   58871   62143   62545   63080   63369   63905   64445   72442   74013
   78149   80066   84914   86174   89943   91254   91924   93146   96194
   98492  106299  108510  108981  109785  109801  114903  114940  115838
  116207  116350  120685  120799  123671  128584  135178  144067  144286
  144550  147852  149021  150365  153171  155248  155734  155777  156939
  158263  161252  163902  164122  164390  168549  170972  171772  172510
  177847  179260  179277  180079  182295  184031  184182  184527  185367
  186235  186339  188449  189081  189998  190693  192374  192654  194715
  195974  198995  203220  207975  218722  219190  219848  224701  227344
  233623  236831  237555  240474  242116  243723  2

In [ ]:
len(users_with_high_clicks)

588

In [ ]:
import plotly.express as px

In [ ]:
train_data_10_10_24_10_11_24_final = pd.read_parquet('/content/drive/MyDrive/wb_data/train_data_10_10_24_10_11_24_final.parquet', engine='pyarrow')
train_data_10_10_24_10_11_24_final.shape

(58872076, 5)

In [ ]:
max_nm_id = text_data_69020_final['nm_id'].max()
min_nm_id = text_data_69020_final['nm_id'].min()

print(f"Максимальное значение nm_id: {max_nm_id}")
print(f"Минимальное значение nm_id: {min_nm_id}")

In [ ]:
train_data_10_10_24_10_11_24_final

In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd

In [ ]:
text_data_69020_final = pd.read_parquet('/content/drive/MyDrive/wb_data/text_data_69020_final.parquet', engine='pyarrow')
text_data_69020_final.shape

NameError: name 'pd' is not defined

In [ ]:
text_data_69020_final.head()

In [ ]:
text_data_69020_final.loc[text_data_69020_final['nm_id'] == 33500, 'characteristics'].values[0]

In [ ]:
import subprocess

directory_path = 'clip_rerank/images-2'

command = f"ls -1 {directory_path} | wc -l"
result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
count = result.stdout.decode().strip()

print(f"{count}")

<div class="alert alert-info">
По количеству данных можно судить о возможности применеия моделей использующих нейросети. Особенно учитывая количество изображений 419499
</div>

In [ ]:
text_data_69020_final.head()

In [ ]:
text_data_69020_final[text_data_69020_final['characteristics'].notna()]

In [ ]:
text_data_69020_final = text_data_69020_final.drop(['characteristics'], axis=1)

<div class="alert alert-info">
Столбец characteristics не содержит данных.
</div>

# <a id="eda"> </a>$\text{Первичные разведочный анализ данных (EDA)}$

In [ ]:
text_data_69020_final['description'].str.len().max()

Максимальная длинна строки описания 5000 символов

<div class="alert alert-info">
В поляз title, brandname, description  присутствует не только русский текст. Для построения эмбедингов целесообразно использование моделей использующих несколько языков.
</div>

In [ ]:
import plotly.express as px

<div class="alert alert-info">
По графику видно, что очень мало пользователей покупают много раличных товаров. И большинство пользователей покупают лишь ограниченное количество товаров, чему соответствует длинный хвост.
</div>

<div class="alert alert-info">
Почти все пользователи купили только лишь товар.
</div>

В последний момент произошла проблема с версиями модулей.

In [ ]:
unique_brandname = text_data_69020_final['brandname'].nunique()
unique_mn = text_data_69020_final['nm_id'].nunique()
unique_order_ts = text_data_69020_final["title"].nunique()
print(f"Количество брендов: {unique_brandname}")
print(f"Количество товаров: {unique_mn}")
print(f"Уникальные заголовки: {unique_order_ts}")

In [ ]:
text_data_69020_final['colornames'].explode().nunique()

<div class="alert alert-info">
Отностельно количества товаров количество различных цветов невелико, однако в описаниях на одну картинку иногда приходится больше одного цвета
</div>

In [ ]:
train_data_10_10_24_10_11_24_final["dt"].min(), train_data_10_10_24_10_11_24_final["dt"].max()

<div class="alert alert-info">
Расматриваемый промежуток времени весьма невелик.
</div>

In [ ]:
unique_user = train_data_10_10_24_10_11_24_final['wbuser_id'].nunique()
unique_item = train_data_10_10_24_10_11_24_final['nm_id'].nunique()
print(f"Количество клиентов: {unique_user}")
print(f"Количество товаров: {unique_item}")

<div class="alert alert-danger" role="alert">
 Исправлена формулировака.
 </div>

<div class="alert alert-info">
Количество клиентов привышает количество товаров почти в 10 раз. Найти похожего пользователя из большого их каличество легче, если рассмативать случайную выборку. Однако в предоствленных для анализа данных пользователи схожи по факту покупки платьев, а соотвецтвенно все товары принадлежат одной категории.
</div>

In [ ]:
nm_id_text_data = set(text_data_69020_final['nm_id'].unique())
nm_id_train_data = set(train_data_10_10_24_10_11_24_final['nm_id'].unique())

intersection = nm_id_train_data.intersection(nm_id_text_data)

count_intersection = len(intersection)

print("Количество элементов в пересечении значений nm_id:", count_intersection)

In [ ]:
only_in_text_data = nm_id_text_data - intersection

only_in_train_data = nm_id_train_data - intersection

print("количество товаров, которые есть только в text_data_69020_final:", len(only_in_text_data))
print("ID товаров, которые есть только в train_data_10_10_24_10ы_11_24_final:", only_in_train_data)

При построении рекомндательной системы, которая будет учитывать и схожесть товаров, и схожесть покупок пользователей можно использовать все товары которые есть в датафрейме `text_data_69020_final`.   
Товары описания которых нет, но их покупка осуществленна рассматриваются исключительно как колизия.

Найдем сами изображения описания которых нет.

In [ ]:
import os
from PIL import Image

image_dir = "clip_rerank/images-2"

def load_image_by_id(nm_id, display=False):
    '''
    Функция для получения изображнеия по его nm_id;
    '''
    filename = f"{nm_id}.jpg"
    image_path = os.path.join(image_dir, filename)


    if os.path.exists(image_path):
        image = Image.open(image_path)
        return image
    else:
        print(f"Изображение с nm_id={nm_id} не найдено.")
        return None

    image = load_image_by_id(nm_id)
    if image:
        image.show()


In [ ]:
load_image_by_id(210699, True)

In [ ]:
load_image_by_id(290074, True)

### Анализ изображений

Изображния можно разделить на несколько категорий:
- изображение с задним фоном
- изображения без фона
- на изображении только товар
- на изображении присутствует текст

<div class="alert alert-info">
Чтобы проанализировать стратификацию изображений по их различному типу можно было бы использовать предобученную модель. Но в контексте постороений рекомндательной системы используюшей encoder для изображений стратификация не сильно важна, поскольку модель будет запоминать осбенности карточек товарах для каждого пользователя.
</div>

In [ ]:
load_image_by_id(7).size

Случайно найденный товар. Платье для ростовой куклы невполне относится к платьям.

In [ ]:
load_image_by_id(108, True)

<div class="alert alert-info">
Все изображения имеют одинаковый формат
</div>

Изученные испочники:

- [поиск по изображению и тексту](https://arxiv.org/pdf/2305.16566)
- [leaPRR model](https://arxiv.org/pdf/2304.12570)
- [семантические ембеденнги](https://arxiv.org/pdf/1707.05612)
- [bottom-up-attention](https://arxiv.org/pdf/1707.07998)
- [Двух-этапная рекомендательная система Статья](https://arxiv.org/pdf/1908.08284)
- [Двух-этапная рекомендательная система Презентация](https://research.zalando.com/fashionxrecsys/workshop-files/presentations/session2-paper4.pdf)
